This notebook extract the model PM composition values to compare with PM literature data of filed campaings in Delhi 2017-2018:

1) Gani, S., Bhandari, S., Seraj, S., Wang, D. S., Patel, K., Soni, P., … Apte, J. S. (2019). Submicron
aerosol composition in the world’s most polluted megacity: the Delhi Aerosol Supersite study.
Atmospheric Chemistry and Physics, 19(10), 6843–6859. https://doi.org/10.5194/acp-19-6843-2019

2) Patel, K., Bhandari, S., Gani, S., Campmier, M. J., Kumar, P., Habib, G., … Hildebrandt Ruiz, L.
(2021). Sources and Dynamics of Submicron Aerosol during the Autumn Onset of the Air Pollution
Season in Delhi, India. ACS Earth and Space Chemistry, 5(1), 118–128.
https://doi.org/10.1021/acsearthspacechem.0c00340

3) Gunthe, S.S., Liu, P., Panda, U. et al. Enhanced aerosol particle growth sustained by high continental
chlorine emission in India. Nat. Geosci. 14, 77–84 (2021). https://doi.org/10.1038/s41561-020-00677-x

In [1]:
#import modules
%run ../load_main.py
from WRFChemToolkit.analysis import aerosols_202 as aer, plots as pt, statistics as st, IGP as igp
from Air_Quality_notebooks import utils as utl

WRFchem outputs data path is: dpath=/geos/d21/s1878599/IGP


In [2]:
#Load data for each season
post_pth=dpath+'/outputs/autumn/wrfout*.nc'
win_pth= dpath+'/outputs/winter/wrfout*.nc'
mon_pth= dpath+'/outputs/monsoon/wrfout*.nc'
pre_pth= dpath+'/outputs/premonsoon/wrfout*.nc'

pth_d={"pre":pre_pth,"mon":mon_pth,"post":post_pth,"win":win_pth}

s=utl.dictdata(pth_d)

In [3]:
#Get aerosols for each seasons
def get_aer(d):
    aer_IGP={}
    for s, val in d.items():
        aer_IGP.update({s :aer.get_aerosols(val)})
    return aer_IGP

In [4]:
aer=get_aer(s)

In [5]:
# add PM1
for s,ds in aer.items():
    ds['pm1_POA']=ds.oc_a01/ds.ALT
    ds['pm1_SOA'] = (ds.glysoa_r1_a01 +ds.glysoa_r2_a01+ds.glysoa_sfc_a01+ \
    ds.glysoa_oh_a01+ds.glysoa_nh4_a01+ds.asoaX_a01+ \
    ds.asoa1_a01 + ds.asoa2_a01+ ds.asoa3_a01+ds.asoa4_a01+ \
    ds.bsoa1_a01 + ds.bsoa2_a01+ ds.bsoa3_a01+ds.bsoa4_a01+ \
    ds.bsoaX_a01)/ds.ALT
    ds['pm1_no3']=ds.no3_a01/ds.ALT
    ds['pm1_sulf']=ds.so4_a01/ds.ALT
    ds['pm1_amm']=ds.nh4_a01/ds.ALT
    ds['pm1_cl']=ds.cl_a01/ds.ALT
    ds['pm1_na']=ds.na_a01/ds.ALT
    ds['pm1_bc']=ds.bc_a01/ds.ALT
    ds['pm1_oth']=ds.oin_a01/ds.ALT
    ds['pm1_water']=ds.water_a01/ds.ALT

In [6]:
# get time average of aerosols
def get_taer(d):
    aer_IGP={}
    for s, val in d.items():
        aer_IGP.update({s :st.time_mean(val,'Time')})
    return aer_IGP

t_aer=get_taer(aer)

In [7]:
#get relevant OA data from model
def get_pm1(ds):
    data ={"pm1_no3" : (["lat","lon"], ds.pm1_no3.values[0,:,:]),
           "pm1_sulf" : (["lat","lon"], ds.pm1_sulf.values[0,:,:]),
           "pm1_amm" : (["lat","lon"], ds.pm1_amm.values[0,:,:]),
           "pm1_cl" : (["lat","lon"], ds.pm1_cl.values[0,:,:]),
           "pm1_na" : (["lat","lon"], ds.pm1_na.values[0,:,:]),
           "pm1_bc" : (["lat","lon"], ds.pm1_bc.values[0,:,:]),
           "pm1_oth" : (["lat","lon"], ds.pm1_oth.values[0,:,:]),
           "pm1_SOA" : (["lat","lon"], ds.pm1_SOA.values[0,:,:]),
           "pm1_POA" : (["lat","lon"], ds.pm1_POA.values[0,:,:]),
           "pm1_water" : (["lat","lon"], ds.pm1_water.values[0,:,:])
          }
    coords={'lat': (['lat'], ds.XLAT[0,:,0]), 'lon': (['lon'], ds.XLONG[0,0,:])}
    oa = xr.Dataset(data, coords=coords)
    return oa

p={}
for k,v in t_aer.items():
    p.update({k:get_pm1(v)})

#put in a dictionary by season
pm1={"premonsoon": p["pre"],"monsoon":p["mon"],"postmonsoon": p["post"],"winter": p["win"]}

In [8]:
def get_pm25(ds):
    data ={"pm25_no3" : (["lat","lon"], ds.pm25_no3.values[0,:,:]),
           "pm25_sulf" : (["lat","lon"], ds.pm25_so4.values[0,:,:]),
           "pm25_amm" : (["lat","lon"], ds.pm25_nh4.values[0,:,:]),
           "pm25_cl" : (["lat","lon"], ds.pm25_cl.values[0,:,:]),
           "pm25_na" : (["lat","lon"], ds.pm25_na.values[0,:,:]),
           "pm25_bc" : (["lat","lon"], ds.pm25_bc.values[0,:,:]),
           "pm25_oth" : (["lat","lon"], ds.pm25_oin.values[0,:,:]),
           "pm25_SOA" : (["lat","lon"], ds.pm25_SOA.values[0,:,:]),
           "pm25_POA" : (["lat","lon"], ds.pm25_POA.values[0,:,:]),
           "pm25_water" : (["lat","lon"], ds.pm25_water.values[0,:,:])
          }
    coords={'lat': (['lat'], ds.XLAT[0,:,0]), 'lon': (['lon'], ds.XLONG[0,0,:])}
    oa = xr.Dataset(data, coords=coords)
    return oa

p={}
for k,v in t_aer.items():
    p.update({k:get_pm25(v)})

#put in a dictionary by season
pm25={"premonsoon": p["pre"],"monsoon":p["mon"],"postmonsoon": p["post"],"winter": p["win"]}

In [9]:
# function for extracting data at a defined location.
def locate(d,lat,lon):
    city={}
    for s, val in d.items():
        city.update({s : val.sel(lat=lat,lon=lon,method='nearest')})
    return city

### Get data at site locations

In [10]:
# Indian Meteorological Department campus (28.59° N, 77.22° E; 200 m above sea level) during February–March 2018

In [11]:
delhi_pm1=locate(pm1,28.59, 77.22)

In [12]:
delhi_pm1

{'premonsoon': <xarray.Dataset>
 Dimensions:    ()
 Coordinates:
     lat        float32 28.635994
     lon        float32 77.176445
 Data variables:
     pm1_no3    float32 1.3288438
     pm1_sulf   float32 0.6682795
     pm1_amm    float32 0.63721114
     pm1_cl     float32 0.0
     pm1_na     float32 0.0005808556
     pm1_bc     float32 1.6482668
     pm1_oth    float32 8.634319
     pm1_SOA    float32 0.5649314
     pm1_POA    float32 1.9935104
     pm1_water  float32 0.0,
 'monsoon': <xarray.Dataset>
 Dimensions:    ()
 Coordinates:
     lat        float32 28.635994
     lon        float32 77.176445
 Data variables:
     pm1_no3    float32 0.34521493
     pm1_sulf   float32 0.54404986
     pm1_amm    float32 0.2929685
     pm1_cl     float32 6.235312e-05
     pm1_na     float32 0.014704865
     pm1_bc     float32 0.6333708
     pm1_oth    float32 3.6659646
     pm1_SOA    float32 1.0653894
     pm1_POA    float32 0.90145224
     pm1_water  float32 0.24802607,
 'postmonsoon': <xarr

In [13]:
delhi_pm25=locate(pm25,28.59, 77.22)

In [14]:
delhi_pm25

{'premonsoon': <xarray.Dataset>
 Dimensions:     ()
 Coordinates:
     lat         float32 28.635994
     lon         float32 77.176445
 Data variables:
     pm25_no3    float32 5.2856216
     pm25_sulf   float32 10.760208
     pm25_amm    float32 5.4573684
     pm25_cl     float32 0.0
     pm25_na     float32 0.15582713
     pm25_bc     float32 9.718925
     pm25_oth    float32 84.85145
     pm25_SOA    float32 5.842578
     pm25_POA    float32 14.541838
     pm25_water  float32 0.0,
 'monsoon': <xarray.Dataset>
 Dimensions:     ()
 Coordinates:
     lat         float32 28.635994
     lon         float32 77.176445
 Data variables:
     pm25_no3    float32 8.301301
     pm25_sulf   float32 7.96247
     pm25_amm    float32 3.7080252
     pm25_cl     float32 0.3041731
     pm25_na     float32 2.3590672
     pm25_bc     float32 3.974393
     pm25_oth    float32 25.867722
     pm25_SOA    float32 13.564557
     pm25_POA    float32 7.5074177
     pm25_water  float32 5.664592,
 'postmonsoon'

In [15]:
 #Delhi Aerosol Supersite is located at the Indian Institute of Technology Delhi (IITD) #28°32′42″N 77°11′32″E

In [16]:
delhi_itt_pm1=locate(pm1,28.54, 77.19)

In [17]:
delhi_itt_pm1

{'premonsoon': <xarray.Dataset>
 Dimensions:    ()
 Coordinates:
     lat        float32 28.453522
     lon        float32 77.176445
 Data variables:
     pm1_no3    float32 1.1008006
     pm1_sulf   float32 0.640723
     pm1_amm    float32 0.5605
     pm1_cl     float32 0.0
     pm1_na     float32 0.0005900841
     pm1_bc     float32 1.2528666
     pm1_oth    float32 6.04013
     pm1_SOA    float32 0.5208318
     pm1_POA    float32 1.6350513
     pm1_water  float32 0.0,
 'monsoon': <xarray.Dataset>
 Dimensions:    ()
 Coordinates:
     lat        float32 28.453522
     lon        float32 77.176445
 Data variables:
     pm1_no3    float32 0.32435223
     pm1_sulf   float32 0.5216729
     pm1_amm    float32 0.27821356
     pm1_cl     float32 5.363794e-05
     pm1_na     float32 0.015289012
     pm1_bc     float32 0.4702625
     pm1_oth    float32 2.5680542
     pm1_SOA    float32 1.0407314
     pm1_POA    float32 0.7839068
     pm1_water  float32 0.25637048,
 'postmonsoon': <xarray.Data

In [18]:
delhi_itt_pm25=locate(pm25,28.54, 77.19)

In [19]:
delhi_itt_pm25

{'premonsoon': <xarray.Dataset>
 Dimensions:     ()
 Coordinates:
     lat         float32 28.453522
     lon         float32 77.176445
 Data variables:
     pm25_no3    float32 4.2712884
     pm25_sulf   float32 10.274269
     pm25_amm    float32 4.978221
     pm25_cl     float32 0.0
     pm25_na     float32 0.15771586
     pm25_bc     float32 7.5719786
     pm25_oth    float32 71.82492
     pm25_SOA    float32 5.304442
     pm25_POA    float32 12.141348
     pm25_water  float32 0.0,
 'monsoon': <xarray.Dataset>
 Dimensions:     ()
 Coordinates:
     lat         float32 28.453522
     lon         float32 77.176445
 Data variables:
     pm25_no3    float32 7.8055415
     pm25_sulf   float32 7.813029
     pm25_amm    float32 3.5226157
     pm25_cl     float32 0.35903913
     pm25_na     float32 2.3753662
     pm25_bc     float32 3.2591705
     pm25_oth    float32 20.511951
     pm25_SOA    float32 13.223827
     pm25_POA    float32 6.92114
     pm25_water  float32 5.641274,
 'postmonsoo